<a href="https://colab.research.google.com/github/payal15604/Sampling_Techniques_CreditCard_Data/blob/main/Sampling_Techniques_on_Unbalanced_Credit_Card_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing necessary libraries
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

In [3]:
# Load the dataset
file_path = "Creditcard_data.csv"
data = pd.read_csv(file_path)

In [5]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,...,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000,772.000000
mean,283.005181,-0.176963,0.217169,0.875172,0.285628,-0.005029,0.159081,0.123329,-0.057547,-0.030384,...,0.004888,-0.096995,-0.040344,-0.002501,0.114337,0.022782,0.023353,-0.017045,68.668290,0.011658
std,171.834196,1.294724,1.173401,1.031878,1.258758,1.098143,1.225682,0.852075,0.830144,0.878183,...,0.609335,0.607228,0.358724,0.621507,0.429667,0.484227,0.300934,0.278332,197.838269,0.107411
min,0.000000,-6.093248,-12.114213,-5.694973,-4.657545,-6.631951,-3.498447,-4.925568,-7.494658,-2.770089,...,-4.134608,-2.776923,-3.553381,-1.867208,-1.389079,-1.243924,-2.377933,-2.735623,0.000000,0.000000
25%,126.500000,-0.896416,-0.174684,0.308677,-0.460058,-0.534567,-0.630717,-0.296289,-0.167880,-0.517068,...,-0.213746,-0.525289,-0.176915,-0.379766,-0.166227,-0.313631,-0.047868,-0.033083,5.987500,0.000000
50%,282.000000,-0.382618,0.285843,0.905435,0.395919,-0.116612,-0.109581,0.116329,0.034755,-0.082270,...,-0.075802,-0.076551,-0.048353,0.091886,0.143723,-0.026414,0.023199,0.021034,16.665000,0.000000
75%,432.000000,1.110739,0.885745,1.532969,1.117559,0.452818,0.482972,0.575390,0.252395,0.412261,...,0.095149,0.307438,0.070085,0.426339,0.425798,0.260408,0.112199,0.087023,55.527500,0.000000
max,581.000000,1.586093,5.267376,3.772857,4.075817,7.672544,5.122103,4.808426,2.134599,5.459274,...,5.273420,1.574750,3.150413,1.215279,1.136720,3.087444,2.490503,1.575380,3828.040000,1.000000


In [7]:
# Class distribution before balancing
print("Class Distribution Before Balancing:")
print(data['Class'].value_counts())

Class Distribution Before Balancing:
Class
0    763
1      9
Name: count, dtype: int64


In [8]:
# Upsampling the minority class
majority = data[data['Class'] == 0]
minority = data[data['Class'] == 1]

minority_upsampled = resample(minority,
                              replace=True,
                              n_samples=len(majority),
                              random_state=42)

In [9]:
# Combine majority and upsampled minority
balanced_data = pd.concat([majority, minority_upsampled])
balanced_data = balanced_data.sample(frac=1, random_state=42)

print("Class Distribution After Balancing:")
print(balanced_data['Class'].value_counts())

Class Distribution After Balancing:
Class
1    763
0    763
Name: count, dtype: int64


In [16]:
balanced_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
623,472,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
77,49,0.921544,-0.067084,0.077461,0.953638,0.067412,0.016152,0.320452,0.038534,-0.391513,...,-0.333930,-1.299920,0.084311,-0.370398,0.155102,-0.834490,0.000909,0.035547,135.51,0
623,472,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
666,503,0.074738,-3.032009,-0.429919,-0.442721,-1.784710,-0.066668,0.098736,-0.222525,-0.241858,...,-0.078927,-1.247242,-0.620402,-0.368545,-0.073359,1.296835,-0.192760,0.121440,712.82,0
623,472,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1


In [17]:
balanced_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1526 entries, 623 to 699
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    1526 non-null   int64  
 1   V1      1526 non-null   float64
 2   V2      1526 non-null   float64
 3   V3      1526 non-null   float64
 4   V4      1526 non-null   float64
 5   V5      1526 non-null   float64
 6   V6      1526 non-null   float64
 7   V7      1526 non-null   float64
 8   V8      1526 non-null   float64
 9   V9      1526 non-null   float64
 10  V10     1526 non-null   float64
 11  V11     1526 non-null   float64
 12  V12     1526 non-null   float64
 13  V13     1526 non-null   float64
 14  V14     1526 non-null   float64
 15  V15     1526 non-null   float64
 16  V16     1526 non-null   float64
 17  V17     1526 non-null   float64
 18  V18     1526 non-null   float64
 19  V19     1526 non-null   float64
 20  V20     1526 non-null   float64
 21  V21     1526 non-null   float64
 22  V22 

In [10]:
# Sampling techniques
sample_1 = balanced_data.sample(frac=0.2, random_state=42)  # Random sampling
sample_2, _ = train_test_split(balanced_data, test_size=0.8, stratify=balanced_data['Class'], random_state=42)  # Stratified sampling

# Systematic sampling
def systematic_sampling(data, step):
    return data.iloc[::step, :]

sample_3 = systematic_sampling(balanced_data, step=5)

# Cluster sampling
clusters = balanced_data.groupby('Time')
sample_4 = clusters.sample(n=1, random_state=42).reset_index(drop=True)

# SMOTE oversampling
X = balanced_data.drop("Class", axis=1)
y = balanced_data["Class"]
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)
sample_5 = pd.concat([pd.DataFrame(X_smote), pd.DataFrame(y_smote, columns=['Class'])], axis=1)


In [11]:
# Display sample sizes
print("Sample Sizes:")
print(f"Sample 1: {len(sample_1)}")
print(f"Sample 2: {len(sample_2)}")
print(f"Sample 3: {len(sample_3)}")
print(f"Sample 4: {len(sample_4)}")
print(f"Sample 5: {len(sample_5)}")

Sample Sizes:
Sample 1: 305
Sample 2: 305
Sample 3: 306
Sample 4: 415
Sample 5: 1526


In [12]:
# Model training and evaluation
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "KNN": KNeighborsClassifier()
}

In [13]:
# Evaluate each model on all samples
rf_acc, lr_acc, svm_acc, dt_acc, knn_acc = [], [], [], [], []
samples = [sample_1, sample_2, sample_3, sample_4, sample_5]

for sample in samples:
    X_sample = sample.drop("Class", axis=1)
    y_sample = sample["Class"]
    X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

    # Random Forest
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    rf_acc.append(accuracy_score(y_test, rf.predict(X_test)))

    # Logistic Regression
    lr = LogisticRegression(max_iter=1000, random_state=42)
    lr.fit(X_train, y_train)
    lr_acc.append(accuracy_score(y_test, lr.predict(X_test)))

    # Support Vector Machine
    svm = SVC(random_state=42)
    svm.fit(X_train, y_train)
    svm_acc.append(accuracy_score(y_test, svm.predict(X_test)))

    # Decision Tree
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train, y_train)
    dt_acc.append(accuracy_score(y_test, dt.predict(X_test)))

    # K-Nearest Neighbors
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    knn_acc.append(accuracy_score(y_test, knn.predict(X_test)))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [14]:
# Compile results
results = pd.DataFrame({
    "Sample": ["Sample1", "Sample2", "Sample3", "Sample4", "Sample5"],
    "RandomForest": rf_acc,
    "LogisticRegression": lr_acc,
    "SVM": svm_acc,
    "DecisionTree": dt_acc,
    "KNN": knn_acc
})

print(results)

    Sample  RandomForest  LogisticRegression       SVM  DecisionTree       KNN
0  Sample1      1.000000            0.950820  0.622951      0.983607  0.901639
1  Sample2      1.000000            0.885246  0.754098      0.983607  0.918033
2  Sample3      0.983871            0.887097  0.758065      0.935484  0.919355
3  Sample4      0.951807            0.951807  0.951807      0.939759  0.951807
4  Sample5      1.000000            0.901961  0.745098      1.000000  0.983660


In [15]:
# Determine the best sampling-model combination
best_sampling_model = results.set_index("Sample").stack().idxmax()
best_accuracy = results.set_index("Sample").stack().max()

print(f"Best Sampling Technique and Model Combination: {best_sampling_model} with Accuracy: {best_accuracy:.2f}")

Best Sampling Technique and Model Combination: ('Sample1', 'RandomForest') with Accuracy: 1.00
